In [1]:
%pwd

'/home/tamerlan/Masters/thesis/yolov5'

In [2]:
import os
import torch
import utils
import cv2
import numpy as np
import matplotlib
from math import ceil

display = utils.notebook_init()  # checks

YOLOv5 🚀 2024-4-24 Python-3.9.19 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 4039MiB)


Setup complete ✅ (8 CPUs, 15.5 GB RAM, 227.3/383.9 GB disk)


In [9]:
def colorize(
    value: np.ndarray, vmin: float = None, vmax: float = None, cmap: str = "afmhot"
):
    if value.ndim > 2:
        return value
    invalid_mask = value == -1

    # normalize
    vmin = value.min() if vmin is None else vmin
    vmax = value.max() if vmax is None else vmax
    value = (value - vmin) / (vmax - vmin)  # vmin..vmax

    # set color
    cmapper = matplotlib.cm.get_cmap(cmap)
    value = cmapper(value, bytes=True)  # (nxmx4)
    value[invalid_mask] = 255
    img = value[..., :3]
    return img

In [ ]:
!python detect.py --weights yolov5s.pt --img 640 --conf 0.50 --source data/images
# display.Image(filename='runs/detect/exp/zidane.jpg', width=600)

In [46]:
!python segment/predict.py --weights yolov5m-seg.pt --img 640 --conf 0.3 --source data/images --save-txt

segment/predict: weights=['yolov5m-seg.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
YOLOv5 🚀 v7.0-304-g22361691 Python-3.9.19 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1050, 4039MiB)

Fusing layers... 
YOLOv5m-seg summary: 301 layers, 21971597 parameters, 0 gradients, 70.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/36 /home/tamerlan/Masters/thesis/yolov5/data/images/cam_01_00001.jpg: 384x640 3 persons, 5 cars, 1 motorcycle, 3 trucks, 225.7ms
image 2/36 /home/tamerlan/Masters/thesis/yolov5/data/images/cam_01_00060.jpg: 384x640 5 persons, 1 bicycle, 3 cars, 1 motorcycle, 2 buss, 

In [44]:
def to_pixel_coords(relative_coords, im_shape_wh):
    return list(round(coord * dimension) for coord, dimension in zip(relative_coords, im_shape_wh))


def get_binary_mask_of_object(segment_line, im_shape_wh, im_shape):
    points = list(map(float, segment_line.split()))
    points.pop(0)
    points = np.array(points)
    points = points.reshape(-1, 2)

    pixels = []

    for point in points:
        pixels.append(to_pixel_coords(point, im_shape_wh))

    pixels = np.array([pixels], dtype=np.int32)

    binary_mask = np.zeros(im_shape, dtype=np.int32)

    binary_mask = np.float32(cv2.fillPoly(binary_mask, pts=pixels, color=255))
    
    binary_mask = cv2.blur(binary_mask, (31, 31))
    _, binary_mask = cv2.threshold(binary_mask, 254, 255, cv2.THRESH_BINARY)
    return binary_mask

def draw_object_vectors(image_path, object_segments_path, depth_pt_path, output_file):
    im = cv2.imread(image_path)
    im_shape = (im.shape)[0:2]


    im_shape_wh = [im_shape[1], im_shape[0]] # w, h

    with open(object_segments_path) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            # if i == 0:
            binary_mask = get_binary_mask_of_object(line, im_shape_wh, im_shape)

            min_depth, max_depth = 1.0, 50.0

            depth = torch.load(depth_pt_path, map_location=torch.device('cpu'))
            depth = depth[0].squeeze().cpu().numpy()

            depth = cv2.resize(depth,
                    (1920,1080), # width, height
                    interpolation=cv2.INTER_CUBIC)

            
            image_array = colorize(depth, min_depth, max_depth)
            binary_mask_3ch = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
            image_array = cv2.bitwise_and(np.float32(image_array), binary_mask_3ch)

            depth_of_object = cv2.bitwise_and(np.float32(depth), binary_mask)

            # убираем все выбросы
            outliers_to_median(depth_of_object)

            # можем искать расположение точки максимума сразу, т.к. все кроме точек объекта - 0
            max_point = np.unravel_index(np.argmax(depth_of_object), depth_of_object.shape) # y, x

            
            # поиск индекса минимума среди элементов depth_of_object 
            # индексы которых совпадают с индексами binary_mask > 0
            min_index = np.argmin(depth_of_object[binary_mask > 0]) 

            # индексы где binary_mask > 0
            indices_nonzero = np.transpose((binary_mask > 0).nonzero())

            # расположение точки минимума на изображении
            min_point = indices_nonzero[min_index]

            im = cv2.arrowedLine(im, (max_point[1], max_point[0]), (min_point[1], min_point[0]), 
                                                color=(0, 0, 255), thickness=3)
            
    cv2.imwrite(output_file, im) 


def find_close_vals(depth_of_object, val, eps=0.3):
    result_ineq = (depth_of_object < val + eps) & (depth_of_object > val - eps) & (depth_of_object != 0)
    indices = np.transpose(result_ineq.nonzero())
    return indices

def outliers_to_median(depth_of_object, percentile=0.01):
    nonzero_points_mask = (depth_of_object != 0)
    nonzero_points = depth_of_object[nonzero_points_mask]
    nonzero_points.sort()

    min_percentile_index = ceil(len(nonzero_points)*percentile) - 1
    min_percentile = nonzero_points[min_percentile_index]

    max_percentile_index = ceil(len(nonzero_points)*(1-percentile)) - 1
    max_percentile = nonzero_points[max_percentile_index]

    median_index = ceil(len(nonzero_points)*(0.5)) - 1
    median = nonzero_points[median_index]

    outliers_mask = nonzero_points_mask & ((depth_of_object <= min_percentile) | (depth_of_object >= max_percentile))
    depth_of_object[outliers_mask] = median


def draw_max_points(image_path, object_segments_path, depth_pt_path, output_file):

    im = cv2.imread(image_path)
    im_shape = (im.shape)[0:2]
    im_shape_wh = [im_shape[1], im_shape[0]] # w, h


    depth = cv2.imread('/home/tamerlan/Masters/thesis/yolov5/tmp/frame_75_depth.png', cv2.IMREAD_GRAYSCALE)
    # depth = torch.load(depth_pt_path, map_location=torch.device('cpu'))
    # depth = depth[0].squeeze().cpu().numpy()

    # print(depth.shape)

    # интерполяция для idisc
    # depth = cv2.resize(depth,
    #         (1920,1080), # width, height
    #         interpolation=cv2.INTER_CUBIC)

    with open(object_segments_path) as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            binary_mask = get_binary_mask_of_object(line, im_shape_wh, im_shape)

            
            #############################
            # создание раскрашенной карты глубины для проверки
            min_depth, max_depth = 1.0, 50.0
            image_array = colorize(depth, min_depth, max_depth)
            binary_mask_3ch = cv2.cvtColor(binary_mask, cv2.COLOR_GRAY2BGR)
            image_array = cv2.bitwise_and(np.float32(image_array), binary_mask_3ch)
            
            print(depth.shape)
            cv2.imwrite(image_array, '/home/tamerlan/Masters/thesis/yolov5/tmp/depth_check.png')

            #############################

            depth_of_object = cv2.bitwise_and(np.float32(depth), binary_mask)

            # убираем все выбросы
            # outliers_to_median(depth_of_object)

            # можем искать расположение точки максимума сразу, т.к. все кроме точек объекта - 0
            max_point = np.unravel_index(np.argmax(depth_of_object), depth_of_object.shape) # y, x
            
            # поиск индекса минимума среди элементов depth_of_object 
            # индексы которых совпадают с индексами binary_mask > 0
            min_index = np.argmin(depth_of_object[binary_mask > 0]) 

            # индексы где binary_mask > 0
            indices_nonzero = np.transpose((binary_mask > 0).nonzero())

            # расположение точки минимума на изображении
            min_point = tuple(indices_nonzero[min_index])

            eps = 1

            max_val = depth_of_object[max_point]
            min_val = depth_of_object[min_point]
            
            max_indices = np.int32(find_close_vals(depth_of_object, val = max_val, eps = eps))
            min_indices = np.int32(find_close_vals(depth_of_object, val = min_val, eps = eps))
            
            overlay = im.copy()
            overlay[max_indices[:,0],max_indices[:,1]] = (255,0,0)
            overlay[min_indices[:,0],min_indices[:,1]] = (0,0,255)

            alpha = 0.4
            
            im = cv2.addWeighted(overlay, alpha, im, 1 - alpha, 0) 
            
    cv2.imwrite(output_file, im)


            


# binary_mask = cv2.threshold(predicted_mask, 0.5, 1, cv2.THRESH_BINARY)[1]

In [29]:
da1 = cv2.imread('/home/tamerlan/Masters/thesis/yolov5/tmp/frame_75_depth.png', cv2.IMREAD_GRAYSCALE)
da1_tensor = torch.from_numpy(da1)
print(da1_tensor.shape)
torch.save(da1_tensor, 'da1_frame75.pt')


# type(da1)

# cv2.imwrite('check_da.png', da1) 


torch.Size([1080, 1920])


In [13]:
image_path = '/home/tamerlan/Masters/thesis/yolov5/data/images/frame_75.jpg'
object_segments_path = '/home/tamerlan/Masters/thesis/yolov5/runs/predict-seg/exp5/labels/frame_75.txt'
# depth_pt_path = '/home/tamerlan/Masters/thesis/yolov5/tmp/depth.pt'
da1_pt_path = '/home/tamerlan/Masters/thesis/yolov5/tmp/da1_frame75.pt'


In [ ]:
draw_object_vectors(image_path, object_segments_path, da1_pt_path, '/home/tamerlan/Masters/thesis/yolov5/tmp/check_da.png')

In [45]:
draw_max_points(image_path, object_segments_path, da1_pt_path, '/home/tamerlan/Masters/thesis/yolov5/tmp/check_da_maxmin.png')

/tmp/ipykernel_20492/2530438966.py:14: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmapper = matplotlib.cm.get_cmap(cmap)


(1080, 1920)
(1080, 1920)
(1080, 1920)
(1080, 1920)
(1080, 1920)
(1080, 1920)
(1080, 1920)
(1080, 1920)
